# solve problem dataset
1. buat dataset baru gbiomass (sum dim nbp)
2. gabungkan dengan dataset glct transisi
3. setelah itu buat dataset selisih baru

tujuan di sini adalah membuat dataset baru dari awal, apakah nilainya berubah atau ngga, karena salah perhitungan fungsinya

In [3]:
import numpy as np
import xarray
import pandas as pd

### mulai dari gbiomass awal

In [2]:
gbiomass_5min = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GBIOMASS.NC", engine="netcdf4")
gbiomass_5min

<xarray.Dataset>
Dimensions:    (NBP: 9, longitude: 4320, latitude: 2160, time: 27)
Coordinates:
  * NBP        (NBP) |S50 b'stems                                            ...
  * longitude  (longitude) float64 -180.0 -179.9 -179.8 ... 179.8 179.9 180.0
  * latitude   (latitude) float64 89.96 89.88 89.79 ... -89.79 -89.87 -89.96
  * time       (time) datetime64[ns] 1970-01-01 1975-01-01 ... 2100-01-01
Data variables:
    GBIOMASS   (time, latitude, longitude, NBP) float32 ...
Attributes:
    modelname:    IMAGE_LPJ
    modelinfo:    image25/MODEL/trunk/model#2844,interface_imagelpj/trunk#170,#0
    conventions:  CF-1.6
    institution:  PBL netherlands Environmental Assessment Agency ( http//www...
    source:       image3.2,interface3.2,lpj3.2
    references:   http//www.pbl.nl/image
    disclaimer:   http//themasites.pbl.nl/models/image/index.php/IMAGE-rights

In [ ]:
gbiomass_5min = gbiomass_5min.drop_sel(NBP=b'long lifetime timber                              ')
gbiomass_5min = gbiomass_5min.drop_sel(NBP=b'short lifetime timber                             ')
gbiomass_5min = gbiomass_5min.drop_sel(NBP=b'charcoal                                          ')
gbiomass_5min = gbiomass_5min.drop_sel(NBP=b'humus                                             ')
gbiomass_5min = gbiomass_5min.drop_sel(NBP=b'litter                                            ')
gbiomass_5min = gbiomass_5min.drop_sel(NBP=b'roots                                             ')

In [ ]:
gbiomass_new = np.zeros((27, 2160, 4320), dtype='float32')

In [ ]:
for time in range(27):
    gbiomass_new[time] = gbiomass_5min['GBIOMASS'].isel(time=time).sum(dim='NBP').values

In [ ]:
gbiomass_new_netcdf = xarray.Dataset({
        "GBIOMASS_new":(["time", "latitude", "longitude"], gbiomass_new)
    },
    coords={
        "time": pd.date_range(start='1970-01-01', end='2100-01-01', freq='5AS'),
        "latitude": gbiomass_5min.coords["latitude"].to_numpy(),
        "longitude": gbiomass_5min.coords["longitude"].to_numpy()
    })
gbiomass_new_netcdf.to_netcdf("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GBIOMASS_new.nc", mode='w', format="NETCDF4")

In [2]:
glct_trans = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GLCT_trans.nc", engine="netcdf4")

In [4]:
gbiomass_new = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GBIOMASS_new.nc", engine="netcdf4")

In [3]:
gbiomass_new.sel(latitude=-22.958333, longitude=-52.041667, method='nearest').to_dataframe()

,GBIOMASS_new,latitude,longitude
time,,,
1970-01-01,25015.539062,-22.958333,-52.041667
1975-01-01,24178.908203,-22.958333,-52.041667
1980-01-01,139.639511,-22.958333,-52.041667
1985-01-01,134.826340,-22.958333,-52.041667
1990-01-01,144.885620,-22.958333,-52.041667
1995-01-01,137.590454,-22.958333,-52.041667
2000-01-01,148.113693,-22.958333,-52.041667
2005-01-01,128.491669,-22.958333,-52.041667
2010-01-01,249.094070,-22.958333,-52.041667


### lalu gabungkan dengan transisi glct, untuk melihat data perbandingannya

In [106]:
merge_data = xarray.merge([gbiomass_new, glct_trans], compat='no_conflicts', join='outer', fill_value=0)

In [ ]:
merge_data.sel(latitude=-3.124, longitude=114.71, method='nearest').to_dataframe()

#### dan membuat data baru gbiomas selisih

In [ ]:
gbiomass_selisih = np.zeros((26, 2160, 4320), dtype='float32')

In [ ]:
for time in range(26):
    gbiomass_selisih[time] = gbiomass_new['GBIOMASS_new'].isel(time=time).values - gbiomass_new['GBIOMASS_new'].isel(time=time+1).values

In [ ]:
gbiomass_subs_new_netcdf = xarray.Dataset({
        "GBIOMASS_subs":(["time", "latitude", "longitude"], gbiomass_selisih)
    },
    coords={
        "time": pd.date_range(start='1975-01-01', end='2100-01-01', freq='5AS'),
        "latitude": gbiomass_5min.coords["latitude"].to_numpy(),
        "longitude": gbiomass_5min.coords["longitude"].to_numpy()
    })
gbiomass_subs_new_netcdf.to_netcdf("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GBIOMASS_selisih26Y.nc", mode='w', format="NETCDF4")

In [27]:
gbiomass_selisih = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GBIOMASS_selisih26Y.nc", engine="netcdf4")
gbiomass_selisih

<xarray.Dataset>
Dimensions:        (time: 26, latitude: 2160, longitude: 4320)
Coordinates:
  * time           (time) datetime64[ns] 1975-01-01 1980-01-01 ... 2100-01-01
  * latitude       (latitude) float64 89.96 89.88 89.79 ... -89.79 -89.87 -89.96
  * longitude      (longitude) float64 -180.0 -179.9 -179.8 ... 179.9 180.0
Data variables:
    GBIOMASS_subs  (time, latitude, longitude) float32 ...

In [3]:
glct5mins_trans = np.zeros((6, 2160, 4320), dtype='<U14')

In [4]:
for i in range(6):
    prev_nicknames_with_to = np.add(glct_trans['GLCT_trans'].isel(time=i), '_to_')
    glct5mins_trans[i] = np.add(prev_nicknames_with_to, glct_trans['GLCT_trans'].isel(time=i+1))

In [5]:
glct_trans_netcdf = xarray.Dataset({
        "GLCT_trans":(["time", "latitude", "longitude"], glct5mins_trans)
    },
    coords={
        "time": pd.date_range(start='1975-01-01', end='2000-01-01', freq='5YS'),
        "latitude": glct_trans.coords["latitude"].to_numpy(),
        "longitude": glct_trans.coords["longitude"].to_numpy()
    })
# glct_trans_netcdf
glct_trans_netcdf.to_netcdf("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GLCT_transto.nc", mode="w", format="NETCDF4")

In [ ]:
glct_transto = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GLCT_transto.nc", engine="netcdf4")
glct_transto

In [30]:
glctdf = pd.read_excel("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GLCT 2.xlsx", index_col=False, engine="openpyxl", sheet_name="Sheet5")

land_use_transitions_postif = dict(glctdf[['Classification_1st', 'Classification_2nd_postif']].values)
land_use_transitions_negatif = dict(glctdf[['Classification_1st', 'Classification_2nd_negatif']].values)
land_use_transitions = dict(glctdf[['trans_name', 'Classification_1st']].values)

In [4]:
glct_lable = glct_transto['GLCT_trans'].to_numpy()
glct_class1st = np.zeros((glct_lable.shape), dtype='<U14') 
glct_class1st = np.array([land_use_transitions.get(label_id, 'ocean_to_ocean') for label_id in glct_lable.ravel()], dtype='<U14').reshape(glct_lable.shape)

In [6]:
trans_glct1st_netcdf = xarray.Dataset({
        "GLCT_trans1st":(["time", "latitude", "longitude"], glct_class1st)
    },
    coords={
        "time": pd.date_range(start='1975-01-01', end='2000-01-01', freq='5AS'),
        "latitude": glct_transto.coords["latitude"].to_numpy(),
        "longitude": glct_transto.coords["longitude"].to_numpy()
    })
# trans_glct1st_netcdf
trans_glct1st_netcdf.to_netcdf("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GLCT_trans1st.nc", mode="w", format="NETCDF4")

In [5]:
glct_1st = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GLCT_trans1st.nc", engine="netcdf4")
glct_1st

<xarray.Dataset>
Dimensions:    (time: 10, latitude: 2160, longitude: 4320)
Coordinates:
  * time       (time) datetime64[ns] 1975-01-01 1980-01-01 ... 2020-01-01
  * latitude   (latitude) float64 89.96 89.88 89.79 ... -89.79 -89.87 -89.96
  * longitude  (longitude) float64 -180.0 -179.9 -179.8 ... 179.8 179.9 180.0
Data variables:
    GLCT_1st   (time, latitude, longitude) <U16 ...

In [11]:
x = [-51.706, -55.974643, 101.457, 109.38, -96.6289, 48.966]
y = [-22.698, -21.204, 31.4420, -7.11, 18.549639, -18.07]
     
gbiomass_new.isel(time=slice(0, 7)).sel(latitude=y[3], longitude=x[3], method='nearest').to_dataframe()

,GBIOMASS_new,latitude,longitude
time,,,
1970-01-01,17052.007812,-7.125,109.374999
1975-01-01,16324.750000,-7.125,109.374999
1980-01-01,16280.656250,-7.125,109.374999
1985-01-01,16057.460938,-7.125,109.374999
1990-01-01,103.810410,-7.125,109.374999
1995-01-01,120.962250,-7.125,109.374999
2000-01-01,119.632912,-7.125,109.374999


In [6]:
gbiomass_new2 = np.zeros((6, 2160, 4320), dtype='float32')
gbiomass_new2 = gbiomass_new['GBIOMASS_new'].isel(time=slice(0, 6)).values #1970 salin

In [ ]:
# trans_filter = ["agri_to_biof", "agri_to_natveg", "biof_to_natveg", "grass_to_biof", "grass_to_natveg", "natveg_to_agri", "natveg_to_biof", "natveg_to_grass"]  

In [8]:
max_year = 6
for i in range(max_year): #1975
    if i <= max_year - 2:
        gbiomass_new2[i+1] = np.where(
        (glct_1st['GLCT_1st'].isel(time=i) == 'agri_to_biof')|
        (glct_1st['GLCT_1st'].isel(time=i) == 'agri_to_natveg')|
        (glct_1st['GLCT_1st'].isel(time=i) == 'biof_to_natveg')|
        (glct_1st['GLCT_1st'].isel(time=i) == 'grass_to_biof')|
        (glct_1st['GLCT_1st'].isel(time=i) == 'grass_to_natveg')|
        (glct_1st['GLCT_1st'].isel(time=i) == 'natveg_to_agri')|
        (glct_1st['GLCT_1st'].isel(time=i) == 'natveg_to_biof')|
        (glct_1st['GLCT_1st'].isel(time=i) == 'natveg_to_grass'),
        gbiomass_new['GBIOMASS_new'].isel(time=i+2), gbiomass_new['GBIOMASS_new'].isel(time=i+1))

In [9]:
gbiomass_new_netcdf = xarray.Dataset({
        "GBIOMASS_new":(["time", "latitude", "longitude"], gbiomass_new2)
    },
    coords={
        "time": pd.date_range(start='1970-01-01', end='1995-01-01', freq='5YS'),
        "latitude": gbiomass_new.coords["latitude"].to_numpy(),
        "longitude": gbiomass_new.coords["longitude"].to_numpy()
    })
gbiomass_new_netcdf
# gbiomass_new_netcdf.to_netcdf("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GBIOMASS_new.nc", mode='w', format="NETCDF4")

<xarray.Dataset>
Dimensions:       (time: 6, latitude: 2160, longitude: 4320)
Coordinates:
  * time          (time) datetime64[ns] 1970-01-01 1975-01-01 ... 1995-01-01
  * latitude      (latitude) float64 89.96 89.88 89.79 ... -89.79 -89.87 -89.96
  * longitude     (longitude) float64 -180.0 -179.9 -179.8 ... 179.8 179.9 180.0
Data variables:
    GBIOMASS_new  (time, latitude, longitude) float32 0.0 0.0 0.0 ... 0.0 0.0

In [12]:
gbiomass_new_netcdf.sel(latitude=y[3], longitude=x[3], method='nearest').to_dataframe()

,GBIOMASS_new,latitude,longitude
time,,,
1970-01-01,17052.007812,-7.125,109.374999
1975-01-01,16324.750000,-7.125,109.374999
1980-01-01,16280.656250,-7.125,109.374999
1985-01-01,103.810410,-7.125,109.374999
1990-01-01,103.810410,-7.125,109.374999
1995-01-01,120.962250,-7.125,109.374999


In [28]:
path = "D:/kerja/asisten riset/vol/milkunarc/cadlan/Analysis1_slidingwindows/GAREA.NC"
garea = xarray.open_dataset(path, engine="netcdf4")

garea_1975 = garea.isel(time=slice(1, 27)) #slice 1975 to 2100
garea_ha = np.nan_to_num(np.multiply(garea_1975['GAREA'].to_numpy(), 100))

# Step 2b: Konversi unit selisih GBIOMASS dari Mg C/ km2 ke selected unit (ton C/ha)
#unit of gbiomass selisih (carbon density) adalah Mg C/ km2
#1 Mg = 1000 kg
#1000kg / 1 Mg = 1000
Mg_to_kg = 1e3
#1 Mg = 1 ton
Mg_to_tonne = 1
#1 km2 = 100 ha
km2_to_ha = 1e2

#Example conversion unit: 20.000 Mg C/km2 x (1 tonne/ 1 Mg)  / (100 ha/ 1km2) --> result in ton C/ha
gbiomass_tonCha = gbiomass_selisih['GBIOMASS_subs'] * Mg_to_tonne / km2_to_ha

#density to total per grid (tonC/ha * ha) --> result tonC
emission_C_ton = np.nan_to_num(np.multiply(gbiomass_tonCha, garea_ha))

ton_to_gigaton = 1e-9

emission_GtTon_CO2 = emission_C_ton * (44/12) * ton_to_gigaton 
emission_CO2_netcdf = xarray.Dataset({
    "emission_CO2":(["time", "latitude", "longitude"], emission_GtTon_CO2)
},coords={
        "time": pd.date_range(start='1975-01-01', end='2100-01-01', freq='5YS'),
        "latitude": garea.coords["latitude"].to_numpy(),
        "longitude": garea.coords["longitude"].to_numpy()
    })
emission_CO2_netcdf

<xarray.Dataset>
Dimensions:       (time: 26, latitude: 2160, longitude: 4320)
Coordinates:
  * time          (time) datetime64[ns] 1975-01-01 1980-01-01 ... 2100-01-01
  * latitude      (latitude) float64 89.96 89.88 89.79 ... -89.79 -89.87 -89.96
  * longitude     (longitude) float64 -180.0 -179.9 -179.8 ... 179.8 179.9 180.0
Data variables:
    emission_CO2  (time, latitude, longitude) float32 0.0 0.0 0.0 ... 0.0 0.0

In [ ]:
# Membuat indeks untuk kondisi gbiomas > 0
gbiomas_condition = emission_CO2_netcdf['emission_CO2'].isel(time=slice(0, 6)).values > 0
glct_lable = glct_1st['GLCT_trans1st'].to_numpy()
glct_class2nd = np.zeros((glct_lable.shape), dtype='<U14') 

# Memproses dataset berdasarkan kondisi gbiomas > 0
glct_class2nd = np.where(gbiomas_condition, np.array([land_use_transitions_postif.get(label_id, 'ocean_to_ocean') for label_id in glct_lable.ravel()], dtype='<U14').reshape(glct_lable.shape),\
                          np.array([land_use_transitions_negatif.get(label_id, 'ocean_to_ocean') for label_id in glct_lable.ravel()], dtype='<U14').reshape(glct_lable.shape))


In [8]:
trans_glct2st_netcdf = xarray.Dataset({
        "GLCT_trans2st":(["time", "latitude", "longitude"], glct_class2nd)
    },
    coords={
        "time": pd.date_range(start='1975-01-01', end='2000-01-01', freq='5AS'),
        "latitude": glct_1st.coords["latitude"].to_numpy(),
        "longitude": glct_1st.coords["longitude"].to_numpy()
})
# trans_glct2st_netcdf
trans_glct2st_netcdf.to_netcdf("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GLCT_trans2st.nc", mode="w", format="NETCDF4")

In [10]:
trans_glct2st_netcdf.sel(latitude=-21.204, longitude=-55.974, method='nearest').to_dataframe()

,GLCT_trans2st,latitude,longitude
time,,,
1975-01-01,ocean_to_ocean,-21.208333,-55.958334
1980-01-01,LUC Crops Sequ,-21.208333,-55.958334
1985-01-01,Agricultural T,-21.208333,-55.958334
1990-01-01,Agricultural T,-21.208333,-55.958334
1995-01-01,Agricultural T,-21.208333,-55.958334
2000-01-01,Agricultural T,-21.208333,-55.958334


In [34]:
glct_2st = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GLCT_trans2st.nc", engine="netcdf4")
glct_2st

<xarray.Dataset>
Dimensions:        (time: 6, latitude: 2160, longitude: 4320)
Coordinates:
  * time           (time) datetime64[ns] 1975-01-01 1980-01-01 ... 2000-01-01
  * latitude       (latitude) float64 89.96 89.88 89.79 ... -89.79 -89.87 -89.96
  * longitude      (longitude) float64 -180.0 -179.9 -179.8 ... 179.9 180.0
Data variables:
    GLCT_trans2st  (time, latitude, longitude) <U14 ...

In [32]:
kategori = np.array(['Agricultural Transition Emission', 'Forest Harvest', 'LUC Biofuel Emission', 'LUC Crops Emission',
        'LUC Rangeland Emission', 'Land Abandonment Emission', 'Agricultural Transition Sequestration', 'Forest Growth',
        'LUC Biofuel Sequestration', 'LUC Crops Sequestration', 'LUC Rangeland Sequestration', 'Land Abandonment Sequestration'], dtype="<U14")
len(kategori)

12

In [36]:
# glct_akumulasi = np.zeros((6, 2160, 4320), dtype='float32')
glct_occur = np.zeros((12, 6, 2160, 4320), dtype='float32')
kategori = np.array(['Agricultural Transition Emission', 'Forest Harvest', 'LUC Biofuel Emission', 'LUC Crops Emission',
        'LUC Rangeland Emission', 'Land Abandonment Emission', 'Agricultural Transition Sequestration', 'Forest Growth',
        'LUC Biofuel Sequestration', 'LUC Crops Sequestration', 'LUC Rangeland Sequestration', 'Land Abandonment Sequestration'], dtype="<U14")

for idx, kategori in enumerate(kategori):
    for time in range(6):
        glct_occur[idx][time] = np.where(glct_2st['GLCT_trans2st'].isel(time=time) == kategori, emission_CO2_netcdf['emission_CO2'].isel(time=time), 0)

In [37]:
kategori = np.array(['Agricultural Transition Emission', 'Forest Harvest', 'LUC Biofuel Emission', 'LUC Crops Emission',
        'LUC Rangeland Emission', 'Land Abandonment Emission', 'Agricultural Transition Sequestration', 'Forest Growth',
        'LUC Biofuel Sequestration', 'LUC Crops Sequestration', 'LUC Rangeland Sequestration', 'Land Abandonment Sequestration'], dtype="<U38")

glct_occur_netcdf = xarray.Dataset(
    coords={
        "time": pd.date_range(start='1975-01-01', end='2000-01-01', freq='5YS'),
        "latitude": glct_2st.coords["latitude"].to_numpy(),
        "longitude": glct_2st.coords["longitude"].to_numpy()
    })
coords = ("time", "latitude", "longitude")
data_vars = {
    kategori: (coords, glct_occur[i]) for i, kategori in enumerate(kategori)
}

glct_occur_netcdf = glct_occur_netcdf.assign(data_vars)
glct_occur_netcdf
# glct_occur_netcdf.to_netcdf("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GLCT_emisi.nc", mode="w", format="NETCDF4")

<xarray.Dataset>
Dimensions:                                (time: 6, latitude: 2160,
                                            longitude: 4320)
Coordinates:
  * time                                   (time) datetime64[ns] 1975-01-01 ....
  * latitude                               (latitude) float64 89.96 ... -89.96
  * longitude                              (longitude) float64 -180.0 ... 180.0
Data variables:
    Agricultural Transition Emission       (time, latitude, longitude) float32 ...
    Forest Harvest                         (time, latitude, longitude) float32 ...
    LUC Biofuel Emission                   (time, latitude, longitude) float32 ...
    LUC Crops Emission                     (time, latitude, longitude) float32 ...
    LUC Rangeland Emission                 (time, latitude, longitude) float32 ...
    Land Abandonment Emission              (time, latitude, longitude) float32 ...
    Agricultural Transition Sequestration  (time, latitude, longitude) float32 ...
    Forest Growth                          (time, latitude, longitude) float32 ...
    LUC Biofuel Sequestration              (time, latitude, longitude) float32 ...
    LUC Crops Sequestration                (time, latitude, longitude) float32 ...
    LUC Rangeland Sequestration            (time, latitude, longitude) float32 ...
    Land Abandonment Sequestration         (time, latitude, longitude) float32 ...

In [23]:
GLCT_emisi = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GLCT_emisi.nc", engine="netcdf4")
GLCT_emisi['Agricultural Transition Emission']

<xarray.DataArray 'Agricultural Transition Emission' (time: 6, latitude: 2160,
                                                      longitude: 4320)>
[55987200 values with dtype=float32]
Coordinates:
  * time       (time) datetime64[ns] 1975-01-01 1980-01-01 ... 2000-01-01
  * latitude   (latitude) float64 89.96 89.88 89.79 ... -89.79 -89.87 -89.96
  * longitude  (longitude) float64 -180.0 -179.9 -179.8 ... 179.8 179.9 180.0